In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14308,2024-04-07,Libano Divisão 1,15:00,Al-Riyadi,NSA,1.01,28.00,172.5,1.85,1.85,0.0,0.0,0.0,vRD8Tk2A,0.990099,0.035714,0.540541,0.540541,0.025813,122.962,26.020321,0.211613,2.4,1.341641,0.559017,160.46,1.242,0.161307,0.129877,80.0,283.862,114.119560,0.402025,1.2,1.643168,1.369306,324.24,3.732,1.843765,0.494042,-45.0,113,84,1.42,3.86,116.802,592.462,1.315740,0.000000,NaN,0.93,0.186,0.053763,0.000000,0.0,0.000000,6.05,1.210,22.314050,0.00000,0.0,0.00000
14309,2024-04-07,Lituânia Lkl,11:20,Rytas,Lietkabelis,1.37,2.93,169.5,1.87,1.79,0.0,0.0,0.0,hxj3yzLQ,0.729927,0.341297,0.534759,0.558659,0.071224,168.478,99.066446,0.588008,1.8,1.643168,0.912871,103.84,1.836,1.019083,0.555056,33.0,158.056,61.124332,0.386726,1.2,1.643168,1.369306,254.87,2.026,0.866072,0.427479,-44.0,88,77,1.18,3.31,120.618,137.228,0.513064,0.030912,NaN,-0.05,-0.010,-37.000000,0.000000,0.0,0.000000,-0.22,-0.044,-43.863636,0.00000,0.0,0.00000
14310,2024-04-07,Lituânia Lkl,13:30,Siauliai,Pieno Zvaigzdes,1.25,3.69,176.5,1.87,1.83,0.0,0.0,0.0,bFjaxG5K,0.800000,0.271003,0.534759,0.546448,0.071003,209.754,99.796848,0.475780,0.6,1.341641,2.236068,176.00,2.720,1.642605,0.603899,-33.0,408.860,309.425082,0.756800,0.0,0.000000,NaN,281.78,5.282,4.042329,0.765303,-79.0,80,73,2.20,3.86,233.382,415.158,0.698518,0.015289,NaN,-2.02,-0.404,-0.618812,0.000000,0.0,0.000000,-2.47,-0.494,-5.445344,0.00000,0.0,0.00000
14311,2024-04-07,México Cibacopa,00:15,Halcones de Obregon,Caballeros de Culiacan,1.36,2.97,172.5,1.82,1.88,0.0,0.0,0.0,n7JjY1Gk,0.735294,0.336700,0.549451,0.531915,0.071994,141.214,24.068178,0.170438,1.2,1.643168,1.369306,140.39,1.610,0.140535,0.087289,-10.0,200.660,59.641358,0.297226,0.0,0.000000,NaN,275.48,2.522,0.752277,0.298286,-56.0,101,97,1.39,2.84,157.850,281.202,0.525839,0.022933,NaN,1.67,0.334,1.077844,0.000000,0.0,0.000000,-2.81,-0.562,-3.505338,0.00000,0.0,0.00000
14312,2024-04-07,Nova Zelândia Nbl,00:00,Franklin Bulls,Canterbury Rams,1.44,2.65,168.5,1.84,1.86,0.0,0.0,0.0,dYQk4AGU,0.694444,0.377358,0.543478,0.537634,0.071803,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,86,79,1.26,1.25,0.000,0.000,0.418386,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14478,2024-04-07,Chile Lnb,20:00,Sportiva Italiana,Liceo Pablo Neruda,1.20,4.09,145.5,1.82,1.88,0.0,0.0,0.0,GIoEJuJ2,0.833333,0.244499,0.549451,0.531915,0.077832,258.666,105.597369,0.408238,1.2,1.643168,1.369306,269.80,3.386,1.566439,0.462622,-22.0,387.626,233.124853,0.601417,0.6,1.341641,2.236068,756.60,5.804,2.903813,0.500312,-104.0,95,78,2.84,9.70,204.620,424.638,0.772604,0.022933,NaN,1.37,0.274,0.729927,0.000000,0.0,0.000000,-5.00,-1.000,-3.090000,0.00000,0.0,0.00000
14479,2024-04-07,Coreia Do Sul Kbl,06:00,Suwon KT,Mobis Phoebus,1.45,2.72,177.5,1.80,1.86,0.0,0.0,0.0,pxU3461d,0.689655,0.367647,0.555556,0.537634,0.057302,149.900,55.100710,0.367583,1.8,1.643168,0.912871,128.34,1.648,0.520260,0.315691,20.0,219.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
14341,04:05,Japão B.League,Brave Thunders,Hiroshima D.,1.75,Back Home
14376,11:00,Rússia Liga Vtb United,Runa Basket Moscow,MBA Moscow,1.78,Back Home
14377,15:00,Sérvia Primeira Liga,OKK Novi Pazar,Sloga,1.92,Back Home
14379,12:30,Sérvia 1. Zls Feminino,Novosadska Novi Sad F,ZKK Kraljevo F,2.68,Back Home
14392,08:00,Suécia Superettan,Tureberg,Malbas,1.55,Back Home
